In [15]:
import torch
from oxdna_torch import load_system, OxDNAEnergy, write_configuration
from oxdna_torch.integrator import LangevinIntegrator
import oxDNA_analysis_tools as oat
from oxDNA_analysis_tools.UTILS.RyeReader import describe, get_confs
from oxDNA_analysis_tools.UTILS.oxview import from_path
from oxdna_torch.io import write_configuration

In [20]:
# Set device (use 'cuda' if a GPU is available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load system files
# Replace "hairpin.top" and "hairpin.conf" with your actual file paths
topology, state = load_system("HAIRPIN/initial.top", "HAIRPIN/initial.conf", device=device)

print(f"Loaded system with {topology.n_nucleotides} nucleotides.")

Loaded system with 18 nucleotides.


In [3]:
# from oxDNA_analysis_tools.UTILS.oxview import from_path
# from_path("HAIRPIN/initial.top", "HAIRPIN/initial.conf")

In [21]:
# 2. Initialize the Energy Model
# Temperature is provided in oxDNA reduced units (T_kelvin / 3000)
model = OxDNAEnergy(
    topology, 
    temperature=0.1113, # ~334K
    seq_dependent=True
).to('cuda')

# 3. Compute Energy and Forces
# Positions and quaternions in 'state' can track gradients
energy = model(state)
forces = model.compute_forces(state)

print(f"Total Potential Energy: {energy.item()}")

# 4. Run Dynamics
integrator = LangevinIntegrator(
    model, 
    dt=0.003, 
    gamma=1.0, 
    temperature=0.1113
)

Total Potential Energy: -6.738764838556694


In [27]:
n_steps=20
trajectory = integrator.rollout(state, n_steps=n_steps)

In [28]:
write_configuration(
    "last_conf.dat",
    trajectory[-1],
    topology,
    n_steps
)

In [29]:
from_path("HAIRPIN/initial.top", "last_conf.dat", **{"inbox_settings":['None','None']})